In [1]:
from medtracker import *
from medtracker import database

/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:14: ExtDeprecationWarning: Importing flask.ext.mail is deprecated, use flask_mail instead.
  from flask.ext.mail import Mail
/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:15: ExtDeprecationWarning: Importing flask.ext.login is deprecated, use flask_login instead.
  import flask.ext.login as flask_login


In [2]:
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.dates as mdates

import pandas as pd
import numpy as np

import datetime
import plotly.graph_objects as go

from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings('ignore')

In [3]:
patients = models.Patient.query.all()
devices = models.Device.query.all()

In [4]:
print(len(patients))
print(len(devices))

3
3


In [5]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)

In [6]:
def number_complete(patients,last_days):
    start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=last_days-1)+datetime.timedelta(days=1)).date()
    outcount = 0
    for p in patients:
        if len(p.surveys.filter(models.SurveyResponse.start_time < start_date).all())>0:
            outcount += 1
    return outcount

In [ ]:
df = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
df

In [143]:
def plotlyBarplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Dark2").colors],
                    stacked=False,percent=False,ordered=False,xtype="category",grouped=False,order2=True,show_legend=False):
    yaxis=go.layout.YAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    xaxis=go.layout.XAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type=xtype
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    fig.update_layout(xaxis_type = xtype)
    ylab = y
    xlab = x
    
    ix = 0
    catorder = list(data.groupby(xlab).sum().index)
    if hue != None:
        #data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            totalcounts = data.groupby(xlab).sum()[ylab]
        for hue,group in data.groupby(hue):
            counts = group.groupby(xlab).sum()[ylab]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100,1)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=x,y=y,
                           text=y,
                           textposition='auto',
                           orientation='v',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        if grouped:
            counts = data.groupby(xlab).sum()[ylab]
            x = [str(i) for i in counts.index]
        else:
            counts = data[ylab]
            x = [str(i) for i in data[xlab]]
        y = counts.values
        trace = go.Bar(x=x,y=y,
                       text=y,
                       textposition='auto',
                       orientation='v',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if ordered:
        if stacked:
            if percent==False:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'total ascending'})
            else:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'category ascending'})
        else:
            fig.update_layout(xaxis={'categoryorder':'total ascending'})
    else:
        if stacked:
            fig.update_layout(barmode='stack')
    if order2:
        fig.update_layout(xaxis={"categoryorder":"array","categoryarray":catorder})
    fig.update_layout(showlegend=show_legend)
    return fig

In [5]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

In [128]:
from plotly import offline
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

patients = models.Patient.query.all()
devices = models.Device.query.all()

def model_to_pd(model):
    res = [r.to_dict() for r in model.query.all()]
    return pd.DataFrame(res)

sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

devs = pts = model_to_pd(models.Device)
devs_per_day = pd.DataFrame(devs.groupby([devs.creation_time.dt.floor("d")])["creation_time"].count())
devs_per_day.columns = ["daily_new_devices"]
pts = model_to_pd(models.Patient)
pts_per_day = pd.DataFrame(pts.groupby([pts.creation_time.dt.floor("d")])["creation_time"].count())
pts_per_day.columns = ["daily_registered_students"]
res_per_day = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
res_per_day.columns = ["daily_completed_surveys"]
df = pd.merge(pts_per_day,res_per_day,left_index=True,right_index=True,how="outer")
df = pd.merge(df,devs_per_day,left_index=True,right_index=True,how="outer")
begin_time = datetime.datetime.utcnow().date() - datetime.timedelta(days=14)
df = df.reindex(pd.date_range(begin_time, datetime.datetime.utcnow().date())).fillna(0).astype(int)
df["total_registered_students"] = df["daily_registered_students"].cumsum()
df["total_completed_surveys"] = df["daily_registered_students"].cumsum()
df["total_devices"] = df["daily_new_devices"].cumsum()
df.reset_index(inplace=True)
df = df.sort_values(by="index",ascending=True)
df["index"] = [datetime.datetime.strftime(a,"%D") for a in df["index"]]
#plotlyBarplot(data=df,x="index",y="daily_new_devices",width=400,title="New Devices")
fig1 = plotlyBarplot(data=df,x="index",y="total_devices",width=500,height=300,title="Total Devices")
fig2 = plotlyBarplot(data=df,x="index",y="total_registered_students",width=500,height=300,title="Total Registered Students")
#plotlyBarplot(data=df,x="index",y="daily_completed_surveys",width=400,title="Daily Completed Screenings")
df["daily_uncompleted_surveys"] = df["total_registered_students"] - df["daily_completed_surveys"]
df2 = df.loc[:,["index","daily_uncompleted_surveys","daily_completed_surveys"]]
df2.columns = ["index","Not Completed","Completed"]
df2 = df2.melt(id_vars="index")
fig3 = plotlyBarplot(data=df2,x="index",y="value",hue="variable",width=600,height=300, title="Screening Status",stacked=True)
fig3.show()

['05/28/20', '05/29/20', '05/30/20', '05/31/20', '06/01/20', '06/02/20', '06/03/20', '06/04/20', '06/05/20', '06/06/20', '06/07/20', '06/08/20', '06/09/20', '06/10/20', '06/11/20']
['05/28/20', '05/29/20', '05/30/20', '05/31/20', '06/01/20', '06/02/20', '06/03/20', '06/04/20', '06/05/20', '06/06/20', '06/07/20', '06/08/20', '06/09/20', '06/10/20', '06/11/20']


In [129]:
df2

,index,variable,value
0,05/28/20,Not Completed,0
1,05/29/20,Not Completed,0
2,05/30/20,Not Completed,0
3,05/31/20,Not Completed,0
4,06/01/20,Not Completed,0
5,06/02/20,Not Completed,0
6,06/03/20,Not Completed,0
7,06/04/20,Not Completed,0
8,06/05/20,Not Completed,0
9,06/06/20,Not Completed,0


In [ ]:
qres

In [ ]:
%run ~/zhang-ipy/Programs/ipy_setup.py
%matplotlib inline

In [20]:
(datetime.datetime.now()-datetime.timedelta(days=1)).date()

datetime.date(2020, 6, 9)

In [11]:
survey = models.Survey.query.get(1)
sr = survey.responses.filter(models.SurveyResponse.start_time > (datetime.datetime.now()-datetime.timedelta(days=6)).date()).all()
responses = []
for sre in sr: responses.extend([r.to_dict() for r in sre.responses])
qres = responses

qres = pd.DataFrame(qres)
qres["date"] = qres.time.dt.floor('d')
qres = qres.groupby(["date","question_id","uniq_id"]).first()
qres = qres.reset_index()

In [12]:
qres

,date,question_id,uniq_id,id,question_choices,question_title,question_type,response,session_id,survey_id,survey_response_id,survey_title,time,user_id
0,2020-06-08,1,1,1,"{""0"": ""I'm experiencing at least one of these""...",Is this an emergency?,radio,I do not have any of these,XCTNU9I2JKXBLEH727R8XUJJPWH2BFMA,1,1,COVID-19 Screening Tool,2020-06-08 19:11:16.280336,1
1,2020-06-08,4,1,2,"{""0"": ""Temperature >100F"", ""1"": ""Cough"", ""2"": ...",Do you have any of the following symptoms?,select,None of the above,XCTNU9I2JKXBLEH727R8XUJJPWH2BFMA,1,1,COVID-19 Screening Tool,2020-06-08 19:11:18.508105,1
2,2020-06-08,6,1,3,,What is your current body temperature?,text,98.7,XCTNU9I2JKXBLEH727R8XUJJPWH2BFMA,1,1,COVID-19 Screening Tool,2020-06-08 19:11:21.802815,1
3,2020-06-09,1,1,7,"{""0"": ""I'm experiencing at least one of these""...",Is this an emergency?,radio,I do not have any of these,VL2MS23BFLFBWFN2HFC5GTXG3ELGDF3G,1,3,COVID-19 Screening Tool,2020-06-09 16:42:17.443349,1
4,2020-06-09,1,3,10,"{""0"": ""I'm experiencing at least one of these""...",Is this an emergency?,radio,I do not have any of these,N5SQF2UJHHG2DMRYAGNRUW4XJ6CTJEZ3,1,4,COVID-19 Screening Tool,2020-06-09 16:43:57.970524,1
5,2020-06-09,4,1,8,"{""0"": ""Temperature >100F"", ""1"": ""Cough"", ""2"": ...",Do you have any of the following symptoms?,select,None of the above,VL2MS23BFLFBWFN2HFC5GTXG3ELGDF3G,1,3,COVID-19 Screening Tool,2020-06-09 16:42:19.567732,1
6,2020-06-09,4,3,11,"{""0"": ""Temperature >100F"", ""1"": ""Cough"", ""2"": ...",Do you have any of the following symptoms?,select,None of the above,N5SQF2UJHHG2DMRYAGNRUW4XJ6CTJEZ3,1,4,COVID-19 Screening Tool,2020-06-09 16:44:00.805718,1
7,2020-06-09,6,1,9,,What is your current body temperature?,text,97.6,VL2MS23BFLFBWFN2HFC5GTXG3ELGDF3G,1,3,COVID-19 Screening Tool,2020-06-09 16:42:23.791512,1
8,2020-06-09,6,3,12,,What is your current body temperature?,text,99.1,N5SQF2UJHHG2DMRYAGNRUW4XJ6CTJEZ3,1,4,COVID-19 Screening Tool,2020-06-09 16:44:04.832696,1
9,2020-06-10,1,1,13,"{""0"": ""I'm experiencing at least one of these""...",Is this an emergency?,radio,I do not have any of these,P7RC6WX2VXHJA6C396CF5YICAISP9QTA,1,5,COVID-19 Screening Tool,2020-06-10 17:36:40.230348,1


In [38]:
g["count"] = 1

In [70]:
list(pltdf.groupby("date").count().index)

[Timestamp('2020-06-08 00:00:00'),
 Timestamp('2020-06-09 00:00:00'),
 Timestamp('2020-06-10 00:00:00')]

In [108]:
e = [str(a.date()) for a in list(pd.date_range(datetime.datetime.now().date()-datetime.timedelta(days=7),datetime.datetime.now().date()))]

In [127]:
pltdf.groupby("date").sum()["count"]

date
2020-06-03    0.0
2020-06-04    0.0
2020-06-05    0.0
2020-06-06    0.0
2020-06-07    0.0
2020-06-08    1.0
2020-06-09    2.0
2020-06-10    1.0
Name: count, dtype: float64

In [144]:
survey_id = 1
sr = models.Survey.query.get(survey_id).responses
responses = []
for sre in sr: responses.extend([r.to_dict() for r in sre.responses])

qres = pd.DataFrame(responses_last7)
qres["date"] = qres.time.dt.floor('d')
qres = qres.groupby(["date","question_id","uniq_id"]).first()
qres = qres.reset_index()

for n,g in qres.groupby("question_id"):
    title = list(g.question_title)[0]
    choices = list(g.question_choices)[0]
    choices = ast.literal_eval(choices) if choices != "" else {}
    kind = list(g.question_type)[0]
    xtype = "category"
    pltdf = g.loc[:,["date","response"]]
    pltdf["count"] = 1
    pltdf.sort_values("date",inplace=True)
    pltdf["date"] = pltdf["date"].astype(str)
    pltdf.set_index("date",inplace=True)
    for ix in e:
        if ix not in pltdf.index:
            pltdf.loc[ix] = [None,0]
    pltdf.sort_index(inplace=True)
    pltdf.reset_index(inplace=True)
    pltdf=pltdf.fillna(method="bfill")
    fig = plotlyBarplot(data=pltdf,x="date",y="count",hue="response",
                        xtype=xtype,grouped=True,ordered=False,stacked=True,order2=True,show_legend=True)
    figs.append(fig)

In [145]:
for fig in figs:
    fig.show()

In [12]:
plotlyBarplot(data=qres[qres["question_id"]==1],x="question_title",y="response")

In [13]:
from plotly import offline

In [ ]:
df.loc[:,["index","total_registered_students","daily_completed_surveys"]].melt(id_vars="index")

In [ ]:
print(len(patients))
print(number_complete(patients,1))

In [ ]:
def destroy_responses():
    for a in models.SurveyResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.QuestionResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Patient.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Device.query.all():
        db.session.delete(a)
    db.session.commit()

In [14]:
def plotlyCountplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Dark2").colors],
                    stacked=False,percent=False):
    xaxis=go.layout.XAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    yaxis=go.layout.YAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type='category'
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    ylab = y
    xlab = x

    ix = 0
    if hue != None:
        data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            data["count"] = 1
            totalcounts = data.groupby(ylab).count()["count"]
        for hue,group in data.groupby(hue):
            group["count"] = 1
            counts = group.groupby(ylab).count()["count"]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=y,y=x,
                           text=y,
                           textposition='auto',
                           orientation='h',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        data["count"] = 1
        counts = data.groupby(ylab).count()["count"]
        x = [str(i) for i in counts.index]
        y = counts.values
        trace = go.Bar(x=y,y=x,
                       text=y,
                       textposition='auto',
                       orientation='h',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if stacked:
        if percent==False:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'total ascending'})
        else:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'category descending'})
    else:
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig
